In [1]:
# Simple CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class SimpleCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize model with input shape
        model = Sequential()
        input_shape = (height, width, depth)
        chan_dim = -1
        
        # first CONV=>RELU=>BN layer set
        model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # Second CONV=>RELU=>BN layer set
        model.add(Conv2D(64, (3,3), strides=(2,2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # First and only set of FC=>RELU layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        
        return model

In [2]:
# Generate Adversarial image
from tensorflow.keras.losses import MSE
import tensorflow as tf

def generate_image_adversary(model, image, label, eps=2/255.0):
    # cast the image
    image = tf.cast(image, tf.float32)
    
    # record gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tracked for growing gradients
        tape.watch(image)
        
        # use model to make predictions on input image and compute loss
        pred = model(image)
        loss = MSE(label, pred)
        
    # calculate gradients of loss w.r.t image then compute sign of gradient
    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    
    # construct the image adversary
    adversary = (image + (signed_grad * eps)).numpy()
    
    return adversary

In [3]:
# training
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2 as cv

# load mnist dataset and scale the pixel values to range [0, 1]
print('[INFO] loading MNIST dataset...')
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

# add a chennel dimension to images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# one hot encode labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# initialize optimizer and model
print('[INFO] Compiling model...')
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train simple CNN on MNIST
print('[INFO] Training Model...')
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, 
          epochs=10, verbose=1)

# make predictions on testing set for model trained on non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

[INFO] loading MNIST dataset...
[INFO] Compiling model...
[INFO] Training Model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 16s 269us/sample - loss: 0.1995 - accuracy: 0.9404 - val_loss: 0.0671 - val_accuracy: 0.9797
Epoch 2/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0792 - accuracy: 0.9758 - val_loss: 0.0525 - val_accuracy: 0.9834
Epoch 3/10
60000/60000 [==============================] - 10s 171us/sample - loss: 0.0543 - accuracy: 0.9833 - val_loss: 0.0431 - val_accuracy: 0.9865
Epoch 4/10
60000/60000 [==============================] - 10s 167us/sample - loss: 0.0489 - accuracy: 0.9845 - val_loss: 0.0849 - val_accuracy: 0.9722
Epoch 5/10
60000/60000 [==============================] - 10s 173us/sample - loss: 0.0416 - accuracy: 0.9867 - val_loss: 0.0376 - val_accuracy: 0.9867
Epoch 6/10
60000/60000 [==============================] - 10s 172us/sample - loss: 0.0331 - accuracy: 0.9893 - v

In [4]:
# adversary
# loop over sample of images
for i in np.random.choice(np.arange(0, len(testX)), size=(10,)):
    # grab current image and label
    image = testX[i]
    label = testY[i]
    
    # generate an image adversary for current image and make a prediction on adversary
    adversary = generate_image_adversary(model, image.reshape(1,28,28,1), 
                                         label, eps=0.1)
    pred = model.predict(adversary)
    
    # scale both original image and adversary to range [0, 255] and convert them to
    # an unsigned 8-bit integers
    adversary = adversary.reshape((28, 28))*255
    adversary = np.clip(adversary, 0, 255).astype('uint8')
    image = image.reshape((28, 28))*255
    image = image.astype('uint8')
    
    # convert image and adversary from gray scale to three channel to display
    image = np.dstack([image]*3)
    adversary = np.dstack([adversary]*3)
    
    # resize both so we can better visualize them
    image = cv.resize(image, (96, 96))
    adversary = cv.resize(adversary, (96, 96))
    
    # determine the label for both original image and adversarial image
    image_pred = label.argmax()
    adversary_pred = pred[0].argmax()
    color = (0, 255, 0)
    
    if image_pred != adversary_pred:
        color = (0, 0, 255)
        
    # draw predictions on respective output images
    cv.putText(image, str(image_pred), (2, 25), cv.FONT_HERSHEY_SIMPLEX, 
               0.95, (0, 255, 0), 2)
    cv.putText(adversary, str(adversary_pred), (2, 25), cv.FONT_HERSHEY_SIMPLEX, 
               0.95, color, 2)
    
    # stack both images horizontally and display them
    output = np.hstack([image, adversary])
    cv.imshow('FGSM Adversarial Images', output)
    cv.waitKey(0)
    cv.destroyAllWindows()